# Investigating Climate Impacts examples - code examples

## Energy generation



### Solar generation - solar radiation

In [ ]:
# load ukcp solar radiation

In [1]:
# select a site (interactively?)

In [2]:
# create a time series for this location (for each of three eras, 1 per ensemble member)

In [ ]:
# show cange in solar radiation

In [ ]:
# show comparison for 3 sites (SW england, N England, scotland)

### Wind generation - surface wind speed

### Heat waves - health and recreation

holiday related small businesses, how is this affected by temperatures?
higher nighttime temperaturesd also affect leisure sites

impacts on health from min and max temperatures in heat waves - maybe affect the need for cooling in office, or ability to do outdoor work during the day?